# Multilayer Perceptrons with Tensorflow

**XBUS-512: Introduction to AI and Deep Learning**

In this exercise, we will build off our first lab, where use scikit-learn's familiar API to prototype a quick multilayer perceptron, to see how to build an MLP from scratch using the TensorFlow library.


Thanks to [this team](https://github.com/Wall-eSociety/CommentVolumeML) for figuring out the labels for this dataset!

## Imports

In [1]:
import os
import time
import zipfile

import requests
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts

## Download the data

In [2]:
def fetch_data(url, fname):
    """
    Helper method to retreive the data from the UCI ML Repository.
    """
    response = requests.get(url)
    outpath  = os.path.abspath(fname)
    with open(outpath, "wb") as f:
        f.write(response.content)
    
    return outpath

In [3]:
# Fetch and unzip the data

URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00363/Dataset.zip"
ZIPPED_FILES = "facebook_data.zip"
UNZIPPED_FILES = "facebook_data"

zipped_data = fetch_data(URL, os.path.join("..", "fixtures", ZIPPED_FILES))

with zipfile.ZipFile(os.path.join("..", "fixtures", ZIPPED_FILES), "r") as zfiles:
    zfiles.extractall(os.path.join("..", "fixtures", UNZIPPED_FILES))

In [4]:
data = pd.read_csv(
    os.path.join(
        "..", 
        "fixtures", 
        UNZIPPED_FILES, 
        "Dataset", 
        "Training", 
        "Features_Variant_2.csv"
    ),
    header=None
)
data.columns = [
    "likes", "views", "returns", "category", "derived_1", "derived_2", "derived_3",
    "derived_4", "derived_5", "derived_6", "derived_7", "derived_8", "derived_9",
    "derived_10", "derived_11", "derived_12", "derived_13", "derived_14", "derived_15", 
    "derived_16", "derived_17", "derived_18", "derived_19", "derived_20", "derived_21",
    "derived_22", "derived_23", "derived_24", "derived_25", "cc_1", "cc_2", "cc_3",
    "cc_4", "cc_5", "base_time", "length", "shares", "status", "h_local", "sunday_post",
    "monday_post", "tuesday_post", "wednesday_post", "thursday_post", "friday_post",
    "saturday_post", "sunday_base", "monday_base", "tuesday_base", "wednesday_base",
    "thursday_base", "friday_base", "saturday_base", "target"
]

data.describe()

,likes,views,returns,category,derived_1,derived_2,derived_3,derived_4,derived_5,derived_6,...,friday_post,saturday_post,sunday_base,monday_base,tuesday_base,wednesday_base,thursday_base,friday_base,saturday_base,target
count,8.131200e+04,81312.000000,8.131200e+04,81312.000000,81312.000000,81312.000000,81312.000000,81312.000000,81312.000000,81312.000000,...,81312.000000,81312.000000,81312.000000,81312.000000,81312.000000,81312.000000,81312.000000,81312.000000,81312.000000,81312.000000
mean,1.313921e+06,4667.704336,4.475377e+04,24.255633,0.707190,464.665781,55.728933,35.392255,67.588653,0.143361,...,0.146153,0.136954,0.141049,0.133400,0.138417,0.145477,0.155180,0.144997,0.141480,7.190611
std,6.804951e+06,20561.261034,1.109279e+05,19.949156,12.169748,520.925523,85.243275,67.043844,82.836764,7.819979,...,0.353262,0.343801,0.348075,0.340008,0.345340,0.352583,0.362078,0.352100,0.348518,36.049374
min,3.600000e+01,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.673400e+04,0.000000,6.980000e+02,9.000000,0.000000,47.000000,5.190751,2.000000,8.032349,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.929110e+05,0.000000,7.045000e+03,18.000000,0.000000,251.000000,22.794183,13.000000,32.565168,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.204214e+06,99.000000,5.026400e+04,32.000000,0.000000,717.000000,71.791489,42.000000,102.060861,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,4.869723e+08,186370.000000,6.089942e+06,106.000000,1311.000000,2438.000000,1693.500000,1693.500000,743.091650,1311.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1966.000000


In [5]:
def prepare_for_regression(dataframe):
    """
    Prepare the data for a regression problem where we will attempt
    to regress the number of comments that a Facebook post will get
    given other features of the data.
    
    Returns a tuple containing an nd array of features (X)
    and a 1d array for the target (y)
    """
    features = [
        "likes", "views", "returns", "category", "derived_1", "derived_2", "derived_3",
        "derived_4", "derived_5", "derived_6", "derived_7", "derived_8", "derived_9",
        "derived_10", "derived_11", "derived_12", "derived_13", "derived_14", "derived_15", 
        "derived_16", "derived_17", "derived_18", "derived_19", "derived_20", "derived_21",
        "derived_22", "derived_23", "derived_24", "derived_25", "cc_1", "cc_2", "cc_3",
        "cc_4", "cc_5", "base_time", "length", "shares", "status", "h_local", "sunday_post",
        "monday_post", "tuesday_post", "wednesday_post", "thursday_post", "friday_post",
        "saturday_post", "sunday_base", "monday_base", "tuesday_base", "wednesday_base",
        "thursday_base", "friday_base", "saturday_base"     
    ]
    target = "target"
    
    # MLP is sensitive to feature scaling!
    X = MinMaxScaler().fit_transform(dataframe[features].values)
    y = dataframe[target].values
    
    return X, y


def prepare_for_classification(dataframe):
    """
    Prepare the data for a classification problem where we will attempt
    to predict the category of a Facebook post given features of the data.
    
    Returns a tuple containing an nd array of features (X)
    and a 1d array for the target (y)
    """
    features = [
        "likes", "views", "returns", "derived_1", "derived_2", "derived_3",
        "derived_4", "derived_5", "derived_6", "derived_7", "derived_8", "derived_9",
        "derived_10", "derived_11", "derived_12", "derived_13", "derived_14", "derived_15", 
        "derived_16", "derived_17", "derived_18", "derived_19", "derived_20", "derived_21",
        "derived_22", "derived_23", "derived_24", "derived_25", "cc_1", "cc_2", "cc_3",
        "cc_4", "cc_5", "base_time", "length", "shares", "status", "h_local", "sunday_post",
        "monday_post", "tuesday_post", "wednesday_post", "thursday_post", "friday_post",
        "saturday_post", "sunday_base", "monday_base", "tuesday_base", "wednesday_base",
        "thursday_base", "friday_base", "saturday_base", "target"     
    ]
    target = "category"
    
    # MLP is sensitive to feature scaling!
    X = MinMaxScaler().fit_transform(dataframe[features].values)
    y = dataframe[target].values
    
    return X, y

In [6]:
# Prepare the data and break in to training and test splits
X, y = prepare_for_regression(data)
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2, random_state=42)


## Build the architecture

In [7]:
input_layer = Input(shape=(X.shape[1],))
dense_layer_1 = Dense(100, activation="relu")(input_layer)
dense_layer_2 = Dense(50, activation="relu")(dense_layer_1)
dense_layer_3 = Dense(25, activation="relu")(dense_layer_2)
output = Dense(1)(dense_layer_3)

model = Model(inputs=input_layer, outputs=output)
model.compile(
    loss="mean_squared_error", 
    optimizer="adam", 
    metrics=["mean_squared_error"]
)

## Train & evaluate the model

Examine the differences in F1/R2 score and training time

In [8]:
start = time.time()

model.fit(
    X_train, 
    y_train, 
    batch_size=2, 
    epochs=100, 
    verbose=1
)

print("Training took {} seconds".format(
    time.time() - start
))

Epoch 1/100
32525/32525 [==============================] - 99s 3ms/step - loss: 797.0344 - mean_squared_error: 797.0344
Epoch 2/100
32525/32525 [==============================] - 93s 3ms/step - loss: 700.1571 - mean_squared_error: 700.1571
Epoch 3/100
32525/32525 [==============================] - 92s 3ms/step - loss: 629.4226 - mean_squared_error: 629.4226
Epoch 4/100
32525/32525 [==============================] - 92s 3ms/step - loss: 596.6333 - mean_squared_error: 596.6333
Epoch 5/100
32525/32525 [==============================] - 92s 3ms/step - loss: 586.5175 - mean_squared_error: 586.5175
Epoch 6/100
32525/32525 [==============================] - 105s 3ms/step - loss: 570.5201 - mean_squared_error: 570.5201
Epoch 7/100
32525/32525 [==============================] - 105s 3ms/step - loss: 562.3015 - mean_squared_error: 562.3015
Epoch 8/100
32525/32525 [==============================] - 104s 3ms/step - loss: 542.6069 - mean_squared_error: 542.6069
Epoch 9/100
32525/32525 [============

32525/32525 [==============================] - 107s 3ms/step - loss: 365.9749 - mean_squared_error: 365.9749
Epoch 68/100
32525/32525 [==============================] - 102s 3ms/step - loss: 357.8987 - mean_squared_error: 357.8987
Epoch 69/100
32525/32525 [==============================] - 102s 3ms/step - loss: 354.0570 - mean_squared_error: 354.0570
Epoch 70/100
32525/32525 [==============================] - 103s 3ms/step - loss: 361.8463 - mean_squared_error: 361.8463
Epoch 71/100
32525/32525 [==============================] - 113s 3ms/step - loss: 357.6417 - mean_squared_error: 357.6417
Epoch 72/100
32525/32525 [==============================] - 137s 4ms/step - loss: 350.3591 - mean_squared_error: 350.3591
Epoch 73/100
32525/32525 [==============================] - 113s 3ms/step - loss: 360.3193 - mean_squared_error: 360.3193
Epoch 74/100
32525/32525 [==============================] - 91s 3ms/step - loss: 347.8130 - mean_squared_error: 347.8130
Epoch 75/100
32525/32525 [============

In [9]:
pred_train = model.predict(X_train)
print("Training error: {}".format(
    np.sqrt(mean_squared_error(y_train, pred_train))
))

pred = model.predict(X_test)
print("Test error: {}".format(
    np.sqrt(mean_squared_error(y_test, pred))
))

Training error: 17.2599058004813
Test error: 22.179094912040828


## Serialize the model 

In [10]:
RESULTS = os.path.join("..", "results")

if not os.path.exists(RESULTS):
    os.makedirs(RESULTS)
    
filename = os.path.join(RESULTS, "tensorflow_model.h5")
model.save(filename)

## Restore the model and run live predictions